USING QSVC IN ORDER TO TEST STANDARD QUANTUM KERNEL APPROACH

In [1]:
#needed to import qke procedures
import sys
from pathlib import Path

main_dir = str(Path().resolve().parent)
sys.path.insert(1, main_dir)

In [2]:
#scikit learn and tools
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [3]:
#load dataset with panda
#data are scaled outside the notebook
env = pd.read_csv('../data/env.sel3.scaled.csv') 
#take a look of the dataset
env.describe() 

,illuminance,blinds,lamps,rh,co2,temp,occupancy
count,2.865000e+03,2865.000000,2865.000000,2865.000000,2.865000e+03,2.865000e+03,2865.000000
mean,-3.571314e-16,0.801622,0.596464,0.473229,-5.952190e-16,-7.142629e-16,-0.130192
std,1.000000e+00,0.372305,0.489316,0.080185,1.000000e+00,1.000000e+00,0.991662
min,-1.016287e+00,0.000000,0.000000,0.243300,-1.222685e+00,-1.774534e+00,-1.000000
25%,-8.575745e-01,1.000000,0.000000,0.433700,-8.492006e-01,-8.538436e-01,-1.000000
50%,-6.989177e-02,1.000000,1.000000,0.491100,-3.252814e-01,-1.940149e-01,-1.000000
75%,3.357060e-01,1.000000,1.000000,0.524100,7.569402e-01,7.266751e-01,1.000000
max,8.559349e+00,1.000000,1.000000,0.633300,3.583699e+00,2.614091e+00,1.000000


In [4]:
#DEFINE design matrix
#slice dataset
f_rate = 0.05
env_slice = env.sample(frac=f_rate, random_state=123)

Y = env_slice['occupancy']
#X = env[['illuminance', 'blinds','lamps','co', 'rh', 'co2', 'temp']]
X = env_slice[['illuminance', 'blinds','lamps','rh', 'co2', 'temp']]

X.head()

,illuminance,blinds,lamps,rh,co2,temp
1618,-0.064014,1.0,1.0,0.5112,0.513679,-0.894763
610,1.552499,1.0,1.0,0.3178,-0.802352,0.992653
1667,-0.969261,1.0,0.0,0.5350,0.115264,-0.853844
2481,-0.134552,0.0,1.0,0.5418,0.601788,-1.421603
2638,-0.434342,1.0,0.0,0.5088,0.469415,-0.587866


In [5]:
#split design matrix (25% of the design matrix used for test)
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=123)
#sanity check
print(f'TRAIN***** {X_train.shape}')
print(f'TEST***** {X_test.shape}')

TRAIN***** (107, 6)
TEST***** (36, 6)


In [6]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit.circuit import ParameterVector
from qiskit.circuit import QuantumCircuit

NUM_QUBIT =  X_train.shape[1]

# Create 2-qubit feature map
qc = QuantumCircuit(NUM_QUBIT)

# Vectors of input and trainable user parameters
input_params = ParameterVector("x_par", NUM_QUBIT)
#training_params = ParameterVector("θ_par", NUM_QUBIT)
training_params = ParameterVector("θ_par", 1)

# Create an initial rotation layer of trainable parameters
#for i, param in enumerate(training_params):
#    qc.ry(param, qc.qubits[i])

#try with a single paramenter
for i, param in enumerate(input_params):
    qc.ry(training_params[0], qc.qubits[i])



# Create a rotation layer of input parameters
for i, param in enumerate(input_params):
    qc.rz(param, qc.qubits[i])



qc.draw()


┌──────────────┐┌──────────────┐
q_0: ┤ Ry(θ_par[0]) ├┤ Rz(x_par[0]) ├
     ├──────────────┤├──────────────┤
q_1: ┤ Ry(θ_par[0]) ├┤ Rz(x_par[1]) ├
     ├──────────────┤├──────────────┤
q_2: ┤ Ry(θ_par[0]) ├┤ Rz(x_par[2]) ├
     ├──────────────┤├──────────────┤
q_3: ┤ Ry(θ_par[0]) ├┤ Rz(x_par[3]) ├
     ├──────────────┤├──────────────┤
q_4: ┤ Ry(θ_par[0]) ├┤ Rz(x_par[4]) ├
     ├──────────────┤├──────────────┤
q_5: ┤ Ry(θ_par[0]) ├┤ Rz(x_par[5]) ├
     └──────────────┘└──────────────┘

# Andrea's notes on executing TrainableFidelityQuantumKernel on different simulators or real quantum hardware

Both FidelityQuantumKernel and TrainableFidelityQuantumKernel constructors take an optional parameters, namely "fidelity" (default None), which represents an instance of a BaseStateFidelity abstract in qiskit_algoritms.

The library qiskit_algorithm provides an implemenation, "ComputeUncompute", that takes a Sampler (v1) as a parameter.

A different choice in the Sampler (Base, Aer (with or without noise models), qiskit_ibm_runtime) makes the ComputeUncompute, and thus the FidelityQuantumKernel and TrainableFidelityQuantumKernel, exploiting the related simulator or real quantum hardware.

The classes FidelityStatevectorkernel and TrainableFidelityStatevectorkernel, otherwise, exploits only classically simulated statevectors during computations.

An example follows.

## First: build a Sampler and a ComputeUncompute instances
(ComputeUncompute implements BaseStateFidelity abstract)

In [7]:
from qiskit_aer.primitives import Sampler as AerSampler # Aer Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute

my_sampler = AerSampler(run_options={"shots":1024,"seed":123})
my_state_fidelity = ComputeUncompute(sampler=my_sampler)

## Second: build a TrainableFidelityQuantumKernel
which uses our concrete my_state_fidelity object

In [8]:
#define the trainable kernel
#from qiskit_machine_learning.kernels import FidelityStatevectorKernel
#from qiskit_machine_learning.kernels import TrainableFidelityStatevectorKernel
#from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.kernels import TrainableFidelityQuantumKernel


q_kernel = TrainableFidelityQuantumKernel(fidelity=my_state_fidelity, feature_map=qc, training_parameters=training_params)

## Then: all as usual
and thus the same as the qsvc_trained.ipynb notebook

In [9]:
#using callback
from pqk.QKCallback import QKCallback
my_callback = QKCallback();

In [10]:
#get the optimized kernel
from qiskit_machine_learning.kernels.algorithms.quantum_kernel_trainer import QuantumKernelTrainer
from qiskit_algorithms.optimizers import SPSA
from qiskit_machine_learning.utils.loss_functions import SVCLoss


spsa_opt = SPSA(maxiter=2, learning_rate=0.03, perturbation=0.01, termination_checker=my_callback.callback)
loss_func = SVCLoss(C=1.0)

#one initial point per trainable parameter
init_point=[np.pi/2]

qk_trainer = QuantumKernelTrainer(quantum_kernel=q_kernel, loss=loss_func, initial_point= init_point, optimizer=spsa_opt)
qkt_results = qk_trainer.fit(X_train, y_train)

optimized_kernel = qkt_results.quantum_kernel

**********************
Print callback. Iteration 1
Number of function evaluations: 2
The paramenters: [1.57079633]
The function value: 54.58291183015187
The stepsize: 0.0
Whether the step was accepted: True
**********************
**********************
Print callback. Iteration 2
Number of function evaluations: 4
The paramenters: [1.57079633]
The function value: 54.58291183015187
The stepsize: 0.0
Whether the step was accepted: True
**********************


# AAARGH! Except it does not work!!!!

Francesco, can you try the notebook this far?

In [11]:
from qiskit_machine_learning.algorithms.classifiers import QSVC
qsvc = QSVC(quantum_kernel=optimized_kernel)

In [12]:
#trainingprediction...
qsvc.fit(X_train, y_train)

QSVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', max_iter=-1,
     probability=False,
     quantum_kernel=<qiskit_machine_learning.kernels.trainable_fidelity_quantum_kernel.TrainableFidelityQuantumKernel object at 0x75f738ca58e0>,
     random_state=None, shrinking=True, tol=0.001, verbose=False)

In [13]:
from sklearn.metrics import accuracy_score

#result...
predictions = qsvc.predict(X_test)
accuracy_score(predictions, y_test)



0.75